# Using Simplicits Full-Feature API
Users with a physics simulation background might find it useful to use the simplicit's API in a more customizable way. 
In this notebook, we expose the inner details of our mesh-free, geometry-agnostic elastic simulator.

In [ ]:
# Notebook requires k3d
!pip install k3d

## Create an Object
Use an SDF to sample points inside the object.

In [2]:
import os, sys
import logging
import k3d
from functools import partial

import torch 
import numpy as np 
import matplotlib.colors as mcolors

from create_example_object import *

# Import kaolin physics
import kaolin.physics as physics

# Imports for displaying the object
from scipy.spatial import Delaunay
import ipywidgets as widgets
from IPython.display import display

device = 'cuda'
dtype = torch.float32

logging.basicConfig(level=logging.DEBUG, stream=sys.stdout)
logger = logging.getLogger(__name__)

so_pts, _, so_yms, so_prs, so_rhos, so_appx_vol = example_unit_cube_object()
so_pts[:,1] += 1
print(so_appx_vol)

plot = k3d.plot()
plot += k3d.points(so_pts.cpu().detach().numpy(), point_size=0.01)
plot.display()


tensor(1.0000, device='cuda:0')


Output()

# Train Object
Train the object using the following training parameters.

In [3]:
NUM_HANDLES = 5
NUM_STEPS = 10000
LR_START = 1e-3
NUM_SAMPLES = 1000

ENERGY_INTERP_LINSPACE = np.linspace(0, 1, NUM_STEPS, endpoint=False)

so_pts, _, so_yms, so_prs, so_rhos, so_appx_vol = example_unit_cube_object()
so_model = physics.simplicits.network.SimplicitsMLP(spatial_dimensions=3, layer_width=64, num_handles=NUM_HANDLES, num_layers=8)
so_optimizer = torch.optim.Adam(so_model.parameters(), LR_START)

# Don't normalize object to unit cube unless is very small or very lage
# so_bb_pts = (torch.min(so_pts, axis=0).values, torch.max(so_pts, axis=0).values)
# so_normalized_pts = (so_pts - so_bb_pts[0])/(so_bb_pts[1] - so_bb_pts[0])

so_pts = torch.as_tensor(so_pts, device=device, dtype=dtype)
so_yms = torch.as_tensor(so_yms, device=device, dtype=dtype).unsqueeze(-1)
so_prs = torch.as_tensor(so_prs, device=device, dtype=dtype).unsqueeze(-1)
so_rhos = torch.as_tensor(so_rhos, device=device, dtype=dtype).unsqueeze(-1)
so_normalized_pts = so_pts.clone()
so_model.to(device)

partial_train_step = partial(physics.simplicits.train.train_step, 
                             batch_size=10, 
                             num_handles=NUM_HANDLES, 
                             appx_vol=1, 
                             num_samples=NUM_SAMPLES, 
                             le_coeff=1e-1, 
                             lo_coeff=1e6)



so_model.train()
for i in range(NUM_STEPS):
    #Set grads to zero
    so_optimizer.zero_grad()
    #train a step
    le, lo = partial_train_step(so_model, so_normalized_pts, so_yms, so_prs, so_rhos, float(i/NUM_STEPS))
    loss = le + lo
    # Backprop over the losses
    loss.backward()
    # Take optimizer step
    so_optimizer.step()
    
    if i%100 == 0:
        logger.info(f'Training step: {i}, le: {le.item()}, lo: {lo.item()}')

so_model.eval()

INFO:__main__:Training step: 0, le: 332.3600158691406, lo: 60618128.0
INFO:__main__:Training step: 100, le: 6.591569423675537, lo: 159196.640625
INFO:__main__:Training step: 200, le: 13.146135330200195, lo: 148620.671875
INFO:__main__:Training step: 300, le: 75.4280776977539, lo: 80939.875
INFO:__main__:Training step: 400, le: 67.49092102050781, lo: 80609.3671875
INFO:__main__:Training step: 500, le: 82.45319366455078, lo: 81028.4375
INFO:__main__:Training step: 600, le: 69.25288391113281, lo: 80405.28125
INFO:__main__:Training step: 700, le: 148.8509979248047, lo: 42417.10546875
INFO:__main__:Training step: 800, le: 171.318603515625, lo: 40994.40625
INFO:__main__:Training step: 900, le: 175.5380859375, lo: 41235.63671875
INFO:__main__:Training step: 1000, le: 170.42750549316406, lo: 40805.40625
INFO:__main__:Training step: 1100, le: 124.38372039794922, lo: 40845.9453125
INFO:__main__:Training step: 1200, le: 98.15068054199219, lo: 40656.91015625
INFO:__main__:Training step: 1300, le: 

SimplicitsMLP(
  (linear_elu_stack): Sequential(
    (0): Linear(in_features=3, out_features=64, bias=True)
    (1): ELU(alpha=1.0)
    (2): Linear(in_features=64, out_features=64, bias=True)
    (3): ELU(alpha=1.0)
    (4): Linear(in_features=64, out_features=64, bias=True)
    (5): ELU(alpha=1.0)
    (6): Linear(in_features=64, out_features=64, bias=True)
    (7): ELU(alpha=1.0)
    (8): Linear(in_features=64, out_features=64, bias=True)
    (9): ELU(alpha=1.0)
    (10): Linear(in_features=64, out_features=64, bias=True)
    (11): ELU(alpha=1.0)
    (12): Linear(in_features=64, out_features=64, bias=True)
    (13): ELU(alpha=1.0)
    (14): Linear(in_features=64, out_features=64, bias=True)
    (15): ELU(alpha=1.0)
    (16): Linear(in_features=64, out_features=64, bias=True)
    (17): ELU(alpha=1.0)
    (18): Linear(in_features=64, out_features=5, bias=True)
  )
)

## Visualize Learned Weights
Visualizes the various learned skinning eigen-modes/subspaces/bases of the trained object.

In [4]:
def scalar_to_rgb(s_normalized, mincolorstr, maxcolorstr, minval=None, maxval=None):
    cmap = mcolors.LinearSegmentedColormap.from_list("", [mincolorstr, maxcolorstr])

    if(np.sum(s_normalized)==0):
        return cmap(s_normalized)[:, 0:3]*255
    else:
        if minval == None:
            minval = np.min(s_normalized)
        if maxval == None:
            maxval = np.max(s_normalized)
        s_normalized = (s_normalized - np.min(s_normalized)) / (maxval - minval)
        colors_rgb = cmap(s_normalized)[:, 0:3]*255
        return colors_rgb

def rgb2hex(rgb):
    str_res = "0x{0:02x}{1:02x}{2:02x}".format(int(rgb[0]), int(rgb[1]), int(rgb[2]))
    return int(str_res, 16)

modes = so_model(so_pts)

def visualize_mode(viz_mode, modes):
    rgb = scalar_to_rgb(modes[:,viz_mode].detach().cpu().numpy(), "blue", "red")
    colors = [rgb2hex(rgb[xx, :]) for xx in range(rgb.shape[0])]
    return colors

# Viewing 3 of the weight functions
plot = k3d.plot()
plot += k3d.points(so_normalized_pts.detach().cpu().numpy(), colors=visualize_mode(0,modes),  point_size=0.01)
plot += k3d.points(so_normalized_pts.detach().cpu().numpy(), colors=visualize_mode(4,modes),  point_size=0.01)
plot += k3d.points(so_normalized_pts.detach().cpu().numpy(), colors=visualize_mode(2,modes),  point_size=0.01)
plot.display()


Output()

## Simulate Object
Set up the simulator with the following parameters.

In [ ]:
# Sim parameters
NUM_STEPS = 100
DT = 0.05
FLOOR_PLANE = -1
PENALTY_WEIGHT = 10000
NUM_SAMPLES = 1000
device = 'cuda'
dtype = torch.float32
MAX_NEWTON_STEPS=5
MAX_LS_STEPS = 30

# Select the cubature points and corresponding material parameters
sample_indices = torch.randint(low=0, high=so_normalized_pts.shape[0], size=(NUM_SAMPLES,), device=so_normalized_pts.device)
sim_pts = so_pts[sample_indices]
sim_normalized_pts = so_normalized_pts[sample_indices]
sim_yms = 0.1*so_yms[sample_indices]
sim_prs = so_prs[sample_indices]
sim_rhos = 5*so_rhos[sample_indices]
sim_weights = torch.cat((so_model(sim_normalized_pts), torch.ones((sim_normalized_pts.shape[0], 1), device=device)), dim=1)
model_plus_rigid = lambda pts: torch.cat((so_model(pts), torch.ones((pts.shape[0], 1), device=device)), dim=1)

# Initialize simulation DOFs
z = torch.zeros(sim_weights.shape[1]*12 , dtype=dtype, device = device).unsqueeze(-1)
z_prev = z.clone().detach()
z_dot = torch.zeros_like(z, device=device)
x0_flat = sim_pts.flatten().unsqueeze(-1)


M, invM = physics.simplicits.precomputed.lumped_mass_matrix(sim_rhos, so_appx_vol, dim = 3)
dFdz = physics.simplicits.precomputed.jacobian_dF_dz(model_plus_rigid, sim_normalized_pts, z).detach()
dxdz = torch.autograd.functional.jacobian(lambda x: physics.simplicits.utils.weight_function_lbs(sim_pts, tfms = x.reshape(-1,3,4).unsqueeze(0), fcn = model_plus_rigid).flatten(), z.flatten())
bigI = torch.tile(torch.eye(3, device=device).flatten().unsqueeze(dim=1), (NUM_SAMPLES,1)).detach()
B = physics.simplicits.precomputed.lbs_matrix(sim_pts, sim_weights).detach()

# 3*num samples gravities per sample point
grav = torch.tensor([0, 9.8, 0], device=device)

BMB = B.T @ M @ B
BinvMB = B.T @ invM @ B

print(" Density: ",str(sim_rhos[0].item())+"kg/m^3\n", 
      "Youngs Mod: ", str(sim_yms[0].item())+"Pa\n", 
      "Poiss Ratio: ", str(sim_prs[0].item())+"\n", 
      "Appx Vol: ", str(so_appx_vol)+"m^3\n")

In [ ]:
#########################SETUP MATERIAL AND SCENE FORCES######################################################
mus, lams = physics.materials.utils.to_lame(sim_yms, sim_prs)
material_object = physics.materials.NeohookeanMaterial(sim_yms, sim_prs)
gravity_object = physics.utils.Gravity(rhos=sim_rhos, acceleration=grav)
floor_object = physics.utils.Floor(floor_height=FLOOR_PLANE, floor_axis=1)
bdry_cond = physics.utils.Boundary()
bdry_indx = torch.nonzero(sim_pts[:,1]>1.45, as_tuple=False).squeeze()
bdry_pos = sim_pts[bdry_indx,:]
bdry_cond.set_pinned_verts(bdry_indx, bdry_pos)
integration_sampling = torch.as_tensor(so_appx_vol/NUM_SAMPLES, device=device, dtype=sim_pts.dtype)

#######################Physics Energy, Forces, Hessians########################################################
partial_bdry_e = physics.simplicits.simplicits_scene_forces.generate_fcn_simplicits_scene_energy(bdry_cond, B, coeff=PENALTY_WEIGHT, integration_sampling=None)
partial_bdry_g = physics.simplicits.simplicits_scene_forces.generate_fcn_simplicits_scene_gradient(bdry_cond, B, coeff=PENALTY_WEIGHT, integration_sampling=None)
partial_bdry_h = physics.simplicits.simplicits_scene_forces.generate_fcn_simplicits_scene_hessian(bdry_cond, B, coeff=PENALTY_WEIGHT, integration_sampling=None)

partial_grav_e = physics.simplicits.simplicits_scene_forces.generate_fcn_simplicits_scene_energy(gravity_object, B, coeff=1, integration_sampling=integration_sampling) 
partial_grav_g = physics.simplicits.simplicits_scene_forces.generate_fcn_simplicits_scene_gradient(gravity_object, B, coeff=1, integration_sampling=integration_sampling) 
partial_grav_h = physics.simplicits.simplicits_scene_forces.generate_fcn_simplicits_scene_hessian(gravity_object, B, coeff=1, integration_sampling=integration_sampling) 

partial_material_e = physics.simplicits.simplicits_scene_forces.generate_fcn_simplicits_material_energy(material_object, dFdz, coeff=1, integration_sampling=integration_sampling)
partial_material_g = physics.simplicits.simplicits_scene_forces.generate_fcn_simplicits_material_gradient(material_object, dFdz, coeff=1, integration_sampling=integration_sampling)
partial_material_h = physics.simplicits.simplicits_scene_forces.generate_fcn_simplicits_material_hessian(material_object, dFdz, coeff=1, integration_sampling=integration_sampling)

partial_floor_e = physics.simplicits.simplicits_scene_forces.generate_fcn_simplicits_scene_energy(floor_object, B, coeff=PENALTY_WEIGHT, integration_sampling=None)
partial_floor_g = physics.simplicits.simplicits_scene_forces.generate_fcn_simplicits_scene_gradient(floor_object, B, coeff=PENALTY_WEIGHT, integration_sampling=None)
partial_floor_h = physics.simplicits.simplicits_scene_forces.generate_fcn_simplicits_scene_hessian(floor_object, B, coeff=PENALTY_WEIGHT, integration_sampling=None)
###############################################################################

####################Backwards Euler Functions###########################################################
def potential_sum(output, z, z_dot, B, dFdz, x0_flat, bigI, defo_grad_fcns = [], pt_wise_fcns = []):
    # updates the quantity calculated in the output value
    F_ele = torch.matmul(dFdz, z) + bigI
    x_flat = B @ z + x0_flat
    x = x_flat.reshape(-1,3)
    for e in defo_grad_fcns:
        output += e(F_ele)
    for e in pt_wise_fcns:
        output += e(x)
        
def newton_E(z, z_prev, z_dot, B, BMB, dt, x0_flat, dFdz, bigI, defo_grad_energies = [], pt_wise_energies = []):
    pe_sum = torch.tensor([0], device=device, dtype=dtype)
    potential_sum(pe_sum, z, z_dot, B, dFdz, x0_flat, bigI, defo_grad_energies, pt_wise_energies)
    return 0.5 * z.T @ BMB @ z - z.T @ BMB @ z_prev - dt * z.T @ BMB @ z_dot + dt * dt * pe_sum

def newton_G(z, z_prev, z_dot, B, BMB, dt, x0_flat, dFdz, bigI, defo_grad_gradients = [], pt_wise_gradients = []):
    pe_grad_sum = torch.zeros_like(z)
    potential_sum(pe_grad_sum, z, z_dot, B, dFdz, x0_flat, bigI, defo_grad_gradients, pt_wise_gradients)
    return BMB @ z - BMB @ z_prev - dt * BMB @ z_dot + dt * dt * pe_grad_sum

def newton_H(z, z_prev, z_dot, B, BMB, dt, x0_flat, dFdz, bigI, defo_grad_hessians = [], pt_wise_hessians = []):
    pe_hess_sum = torch.zeros(z.shape[0], z.shape[0], device=device, dtype=dtype)
    potential_sum(pe_hess_sum, z, z_dot, B, dFdz, x0_flat, bigI, defo_grad_hessians, pt_wise_hessians)
    return BMB  + dt * dt * pe_hess_sum
    
##########################Backwards Euler Partials#####################################################
partial_newton_E = partial(newton_E, 
                           B=B.detach(), 
                           BMB = BMB.detach(), 
                           dt=DT, 
                           x0_flat=x0_flat.detach(), 
                           dFdz=dFdz.detach(), 
                           bigI=bigI.detach(),
                           defo_grad_energies=[partial_material_e],
                           pt_wise_energies=[partial_grav_e, partial_floor_e, partial_bdry_e])
partial_newton_G = partial(newton_G, 
                           B=B.detach(), 
                           BMB = BMB.detach(), 
                           dt=DT, 
                           x0_flat=x0_flat.detach(), 
                           dFdz=dFdz.detach(), 
                           bigI=bigI.detach(),
                           defo_grad_gradients=[partial_material_g],
                           pt_wise_gradients=[partial_grav_g, partial_floor_g, partial_bdry_g])
partial_newton_H = partial(newton_H, 
                           B=B.detach(), 
                           BMB = BMB.detach(), 
                           dt=DT, 
                           x0_flat=x0_flat.detach(), 
                           dFdz=dFdz.detach(), 
                           bigI=bigI.detach(),
                           defo_grad_hessians=[partial_material_h],
                           pt_wise_hessians=[partial_grav_h, partial_floor_h, partial_bdry_h])


In [ ]:
z = torch.zeros(sim_weights.shape[1]*12 , dtype=dtype, device = device).unsqueeze(-1)
z_prev = z.clone().detach()
z_dot = torch.zeros_like(z, device=device).detach()
x0_flat = sim_pts.flatten().unsqueeze(-1)
states = [z.clone().detach()]
for time_step in range(int(NUM_STEPS)):
    print("Timestep: ", time_step)
    z_prev = z.clone().detach()
    more_partial_newton_E = partial(partial_newton_E, z_prev=z_prev, z_dot=z_dot)
    more_partial_newton_G = partial(partial_newton_G, z_prev=z_prev, z_dot=z_dot)
    more_partial_newton_H = partial(partial_newton_H, z_prev=z_prev, z_dot=z_dot)
    z = physics.utils.optimization.newtons_method(z, more_partial_newton_E, more_partial_newton_G, more_partial_newton_H)
    F_ele = torch.matmul(dFdz, z) + bigI
    x_pts = (B @ z + x0_flat).reshape(-1,3)
    logger.info(f'Floor E:{partial_floor_e(x_pts).item()}, Grav E:{partial_grav_e(x_pts).item()}, Bdry E:{partial_bdry_e(x_pts).item()}, Elastic E:{partial_material_e(F_ele).item()}')
    with torch.no_grad():
        z_dot = (z - z_prev)/DT
    states.append(z.clone().detach())

# Displaying The Simulation (scrub through the timeline slider)

In [ ]:
def generate_ground_plane(plane):
    # Define the size of the ground plane
    x_size = 1
    y_size = 1
    num_points_x = 30
    num_points_y = 30
    
    # Generate grid points
    x = np.linspace(-x_size/2, x_size/2, num_points_x)
    y = np.linspace(-y_size/2, y_size/2, num_points_y)
    X, Y = np.meshgrid(x, y)
    
    # Generate vertices
    vertices = np.column_stack([X.flatten(), plane * np.ones_like(X.flatten()), Y.flatten()])
    vertices += +1e-3*np.random.rand(vertices.shape[0], vertices.shape[1])
    # Perform Delaunay triangulation
    tri = Delaunay(vertices)
    return torch.tensor(vertices), torch.tensor(tri.simplices)

v, f = generate_ground_plane(FLOOR_PLANE)

# Function to create points
def create_points(t):
    z = states[int(t)]
    x = B @ z + x0_flat
    # print(torch.norm(x - x0_flat))
    scene_verts = x.reshape(-1,3).unsqueeze(0).cpu().detach()
    return scene_verts.cpu().detach().numpy()

# Create a plot
plot = k3d.plot(camera_auto_fit=False)
floor_plot = k3d.points(v.cpu().detach().numpy(), point_size=0.02, color=0x00ff00, shader='3d')
plot += floor_plot
# Initial set of points
initial_points = create_points(0)
points_plot = k3d.points(initial_points, point_size=0.1, color=0x00ff00, shader='3d')
plot += points_plot

# Rotate the camera so that y is the vertical axis
plot.camera = [3, 1, 0, 0, 0, 0, 0, 1, 0]  # (eye_x, eye_y, eye_z, target_x, target_y, target_z, up_x, up_y, up_z)


# Display the plot
plot.display()

# Define the function to update the points
def update_points(timestep):
    new_points = create_points(timestep)
    points_plot.positions = new_points.astype(np.float32)

# Create a slider
slider = widgets.FloatSlider(min=0, max=len(states), step=1, value=0)

# Link the slider to the update function
widgets.interactive(update_points, timestep=slider)

# Display the slider
display(slider)